# Real experiment evaluation

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

In [ ]:
def convert_room_to_robot(x_room):
    '''
    Convert coordinates in room reference to robot reference. 
    '''
    assert x_room.shape[0] == 3
    assert x_room.ndim > 1 and x_room.shape[1] >= 1
    
    from math import cos, sin, pi
    
    theta = pi/2.0
    R = np.array([[cos(theta), -sin(theta), 0], [sin(theta), cos(theta), 0], [0, 0, 1]])
    #origin = np.array([3.4, 3.58, 0.0]).reshape((3, 1))
    origin = np.array([3.58, 3.4, 0.0]).reshape((3, 1))
    x_robot = R.dot(x_room - origin)
    return x_robot


datafile = 'experiments/robot_test/circle.csv'
#datafile = 'experiments/robot_test/full_bandlimited1.csv'

anchorsfile = 'experiments/anchors.csv'

tango_system_id = 7585
rtt_system_id = 7592

rtt_counter = -1
tango_counter = -1

def add_name(row):
    global rtt_counter
    global tango_counter 
    if row.system_id == tango_system_id:
        tango_counter += 1
        return 'Tango {}'.format(tango_counter)
    elif row.system_id == rtt_system_id:
        rtt_counter += 1
        return 'RTT {}'.format(rtt_counter)
    
    
def clean_anchor_id(row):
    return row.anchor_id.strip()


data_df = pd.read_csv(datafile)
data_df.loc[:, 'seconds'] = (data_df.timestamp.values - data_df.timestamp.min()) / 1000. # in seconds

anchors_df = pd.read_csv(anchorsfile)
anchors_df.loc[:, 'anchor_id'] = anchors_df.apply(lambda row: clean_anchor_id(row),  axis=1)
anchors_df.loc[:, 'anchor_name'] = anchors_df.apply(lambda row: add_name(row), axis=1) 
anchors_df

In [ ]:
print(data_df.columns)
data_df

In [ ]:

plt.figure()
sns.scatterplot(data=data_df[data_df.system_id==tango_system_id], x='px', y='py', hue='seconds', linewidth=0.0)
sns.scatterplot(data=anchors_df, x='px', y='py', hue='anchor_name', linewidth=0.0, style='system_id', legend=False)
plt.arrow(0, 0, 1, 0, color='red', head_width=0.2)
plt.arrow(0, 0, 0, 1, color='green', head_width=0.2)
delta = 0.1
for i, a in anchors_df.iterrows():
    plt.annotate(a.anchor_name, (a.px+delta, a.py+delta))
plt.legend(loc='lower right', bbox_to_anchor=[1.0, 0.0])
plt.axis('equal')
plt.title(datafile)

In [ ]:
plt.figure()
data = data_df[data_df.system_id==tango_system_id]
sns.scatterplot(data=data, x='seconds', y='px', linewidth=0.0, label='x')
sns.scatterplot(data=data, x='seconds', y='py', linewidth=0.0, label='y')
sns.scatterplot(data=data, x='seconds', y='pz', linewidth=0.0, label='z')
plt.legend()

In [ ]:
def fill_ground_truth(data_df, current_gt=None):
    for i, row in data_df.iterrows():
        if row.system_id == tango_system_id:
            current_gt = [row.px, row.py, row.pz]
        elif current_gt is not None:
            data_df.loc[i, ['px', 'py', 'pz']] = current_gt
        else:
            print(i, 'no gt yet.')
            
fill_ground_truth(data_df)

In [ ]:
def distance_error(row):
    if row.system_id == tango_system_id:
        return 0.0
    anchor_coord = anchors_df.loc[anchors_df.anchor_id==row.anchor_id, ['px', 'py', 'pz']].values.astype(np.float32)
    point_coord = np.array([row.px, row.py, row.pz], dtype=np.float32)
    return np.linalg.norm(point_coord - anchor_coord)
        
data_df.loc[:, 'distance_gt'] = data_df.apply(lambda row: distance_error(row), axis=1)
data_df.loc[:, 'distance_error'] = np.abs(data_df.distance_gt - data_df.distance)

In [ ]:
def plot_distances(data_df):
    import itertools
    colors = itertools.cycle(plt.get_cmap('tab10').colors)
    rtt_ids = anchors_df[anchors_df.system_id==rtt_system_id].anchor_id.unique()
    fig, axs = plt.subplots(1, len(rtt_ids), sharey=True)
    fig.set_size_inches(10, 5)
    for ax, anchor_id in zip(axs, rtt_ids):
        color = next(colors)
        data = data_df[data_df.anchor_id==anchor_id]
        anchor_name = anchors_df.loc[anchors_df.anchor_id==anchor_id, 'anchor_name']
        ax.plot(data.seconds, data.distance_gt, linestyle=':', label=anchor_name, color=color)
        ax.plot(data.seconds, data.distance, linestyle='-', color=color)
        ax.set_ylim(0, 15)
    #sns.scatterplot(data=data_df[data_df.system_id==rtt_system_id], x='seconds', y='distance_error',hue='anchor_id')
plot_distances(data_df)

In [ ]:
parameters = {
    'experiments/robot_test/static1.csv':  np.array([0, 0, 0.2]), # on robot, facing window
    'experiments/robot_test/static2.csv':  np.array([0, 0, 0.1]), # in wooden thing, facing window
    'experiments/robot_test/static3.csv':  np.array([0, 0, 0.1]), # in wooden thing, facing wall
    'experiments/robot_test/static4.csv':  np.array([0, 0, 1.1]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static5.csv':  np.array([0, 0, 1.1]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static6.csv':  np.array([0, 0, 1.1]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static7.csv':  np.array([0, 0, 0.2]), # in wooden thing on stand, facing wall
    'experiments/robot_test/static8.csv':  np.array([0, 0, 0.2]), # in wooden thing on stand, facing wall
}

for i in range(1, 9):
    datafile = 'experiments/robot_test/static{}.csv'.format(i)
    position = parameters[datafile]

    data_df = pd.read_csv(datafile)
    data_df.loc[:, 'seconds'] = (data_df.timestamp.values - data_df.timestamp.min()) / 1000. # in seconds
    fill_ground_truth(data_df, current_gt=position)
    data_df.loc[:, 'distance_gt'] = data_df.apply(lambda row: distance_error(row), axis=1)
    data_df.loc[:, 'distance_error'] = np.abs(data_df.distance_gt - data_df.distance)
    plot_distances(data_df)
    name = datafile.split('/')[-1]
    plt.suptitle(name)

In [ ]:
import os
folder = 'experiments/robot_test/'
file_list = os.listdir(folder)
plt.figure()


end_points_lines_room = np.array([
    [5.046, 5.615, 0.0],
    [4.869, 4.231, 0.0],
    [5.870, 1.830, 0.0],
    [4.254, 1.567, 0.0],
    [2.924, 1.867, 0.0],
    [1.198, 1.416, 0.0]]).T
start_point_room = np.array([1.034, 5.410, 0.0]).reshape((3, 1))

end_points_lines = convert_room_to_robot(end_points_lines_room)
start_point = convert_room_to_robot(start_point_room)

for datafile in sorted(file_list):
    if datafile[-4:] != '.csv':
        continue
    if (datafile[:8] != 'straight') and (datafile[:5] != 'calib') or (datafile[:10] == 'calibratio'):
        continue
    data_df = pd.read_csv(folder + datafile)
    #data_df.loc[:, 'seconds'] = (data_df.timestamp.values - data_df.timestamp.min()) / 1000. # in seconds
    data = data_df[data_df.system_id==tango_system_id]
    sns.scatterplot(data=data, x='px', y='py', label=datafile, linewidth=0.0)

plt.scatter(*start_point[:2], color='black', s=10.0)
for end_point in end_points_lines.T:
    print(end_point)
    plt.scatter(*end_point[:2], color='black', s=5.0)
    plt.plot([start_point[0], end_point[0]], [start_point[1], end_point[1]], color='black')
    
plt.axis('equal')
plt.legend(loc='lower left', bbox_to_anchor=[0.8, 0])
#plt.legend()
#fill_ground_truth(data_df, current_gt=position)

In [ ]:

for datafile in sorted(file_list):
    if datafile[-4:] != '.csv':
        continue
    else:
        data_df = pd.read_csv(folder + datafile)
        data = data_df[data_df.system_id==tango_system_id]
        plt.figure()
        sns.scatterplot(data=data, x='px', y='py', label=datafile, linewidth=0.0)